# Parts Direct (PD) Transaction Dataset

In [1]:
import timeit
tic=timeit.default_timer()
import calendar
import datetime
import numpy as np
import pandas as pd
import snowflake.connector
import pandas.io.sql as sql

con = snowflake.connector.connect(
                user='-----',
                password='-----',
                account='-------'
                )
cursor = con.cursor()


data = """SELECT b.PRT_NO,a.SVC_PRT_STS_DT,a.SVC_UN_NO,a.SO_NO,a.PRT_ORD_QT,a.PRT_ORD_LN_STS_CD,a.prt_ord_ln_no,a.svc_div_no,a.PRT_PRC_LIS_SRC_NO,
a.SVC_PRT_NO,LPAD(a.svc_div_no, 4, '0') || LPAD(a.prt_prc_lis_src_no, 3, '0') ||'_'|| 
a.svc_prt_no ||'_'|| e.CUSTSTATE AS item_id, a.SO_NO || LPAD(a.svc_div_no, 4, '0') || LPAD(a.prt_prc_lis_src_no, 3, '0') AS so_div_pls,
a.PRT_DS,a.ASL_CATEGORY,e.SERVICEORDER,e.CUSTSTATE,b.PRT_ORD_LN_QT,b.TOT_ORD_LN_AM,b.PRT_DIV_NO
FROM
PRD_PARTSDIRECT.BATCH.PRTDTPL_PRT_LN b
JOIN PRD_NFDT.NFDT_PERM_TBLS.SERVICEORDER_PARTS a 
ON a.svc_prt_no=b.PRT_NO
JOIN PRD_NFDT.NFDT_PERM_TBLS.TRANSACTIONS e
ON e.SERVICEORDER=a.SO_NO
WHERE a.ASL_CATEGORY!='TECH'
AND
a.SVC_PRT_STS_DT >= '2020-01-01'
"""
#Extract and load PD Transaction data from Snowflake database.
query = pd.read_sql_query(data, con)

#Filter data for 20 states where the Home Town stores are present
query = query[query["CUSTSTATE"].isin(['GA','WI','FL','TX','LA','ND','CA','AR','CO','TN',
                                        'KS','NM','MO','NE','AZ','IL','OK','IA','NV','UT'])]
query = query[~query["SVC_DIV_NO"].isin(['099'])]
query['SVC_PRT_STS_DT'] = pd.to_datetime(query['SVC_PRT_STS_DT'], format='%Y-%m-%d', infer_datetime_format=True)
query['Year'] = query['SVC_PRT_STS_DT'].dt.year
query['Month'] = query['SVC_PRT_STS_DT'].dt.month
query['monthName'] = query['Month'].apply(lambda x: calendar.month_name[x])
query['Month_Name'] = query['monthName'].str[:3]
query['Month_Year'] = query['Month_Name'].map(str) + "_" + query['Year'].map(str)

data = query[['PRT_NO','PRT_DIV_NO','PRT_ORD_LN_QT','TOT_ORD_LN_AM','CUSTSTATE','PRT_DS','Month_Year']]
data = data.groupby(['PRT_NO','PRT_DIV_NO','CUSTSTATE','PRT_DS','Month_Year']).agg(
    Qnt_Sold=pd.NamedAgg(column='PRT_ORD_LN_QT', aggfunc='sum'),
    Revenue=pd.NamedAgg(column='TOT_ORD_LN_AM', aggfunc='sum')
)
data.reset_index(drop=False, inplace=True)

#Load and join tables needed for parts division description. 
conn = snowflake.connector.connect(
                user='bagabi',
                password='Ben@2020',
                account='sears_hs_prod.us-east-1'
                )
cursor = conn.cursor()
data2 = """SELECT a.svc_div_no,a.SVC_PRT_NO,LPAD(a.svc_div_no, 4, '0') || LPAD(a.prt_prc_lis_src_no, 3, '0') || 
a.svc_prt_no AS RECOMMENDED_PARTID, LPAD(a.svc_div_no, 4, '0') AS ID,
a.PRT_DS
FROM
PRD_NFDT.NFDT_PERM_TBLS.SERVICEORDER_PARTS a
Where a.SVC_PRT_STS_DT >= '2020-01-01'
"""
dat = """SELECT ID, NAME from PRD_KM.KM.DIVISION
"""
div = pd.read_sql_query(dat, conn)
query2 = pd.read_sql_query(data2, conn)

query2_div = pd.merge(query2, div)
div_desc = query2_div[['SVC_PRT_NO','NAME']]
div_desc.rename({'SVC_PRT_NO': 'PRT_NO'}, axis=1, inplace=True)

desc_div = pd.merge(data, div_desc)
desc_div2 = desc_div.drop_duplicates(subset=['PRT_NO','CUSTSTATE','Month_Year'], keep='first')

dy = desc_div2.copy()
dy['Month_Year'] = dy['Month_Year'].apply(lambda x: x.replace('_','')).map(str)
dy['YEAR'] = dy['Month_Year'].str[-4:]
dy['MONTH'] = dy['Month_Year'].str[:3]
dy = dy[['PRT_NO','PRT_DS','PRT_DIV_NO','NAME','Qnt_Sold','Revenue','CUSTSTATE','MONTH','YEAR']]
dy.rename({'NAME':'DIVIDION_DESCRIPTION','Qnt_Sold':'QNT_SOLD','Revenue':'REVENUE'}, axis=1, inplace=True)
dy['REVENUE'] = dy['REVENUE'].map('${:,.2f}'.format)
dy.sort_values("QNT_SOLD", inplace = True, ascending=False)
#dy.to_excel('Home Town Commercial Business Top Selling Parts.xlsx',index=False)

toc=timeit.default_timer()
Elapsed = toc - tic 
hours, rem = divmod(Elapsed, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

C:\Users\bagabi\AppData\Local\Continuum\anaconda3\lib\site-packages\snowflake\connector\options.py:78: UserWarning: You have an incompatible version of 'pyarrow' installed (0.15.1), please install a version that adheres to: 'pyarrow<0.18.0,>=0.17.0; extra == "pandas"'
  warn_incompatible_dep('pyarrow', _installed_pyarrow_version.version, _expected_pyarrow_version)
C:\Users\bagabi\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


00:02:46.60


In [2]:
dy.head()

,PRT_NO,PRT_DS,PRT_DIV_NO,DIVIDION_DESCRIPTION,QNT_SOLD,REVENUE,CUSTSTATE,MONTH,YEAR
289802,799866,CARBURETOR,71,Lawn & Garden/Yard/Farm,33870,"$1,473,371.75",TX,Jan,2021
564954,W10193886,TUB-OUTER,026,Laundry Appliances,4206,"$999,914.19",CA,Aug,2020
74825,280184,SUPPORT,026,Laundry Appliances,3157,"$403,767.54",CA,Jul,2020
76312,280184,SUPPORT,026,Laundry Appliances,2009,"$256,942.98",CA,Jun,2020
568814,W10193886,TUB-OUTER,026,Laundry Appliances,1402,"$333,304.73",TX,Aug,2020


# Technicians Dataset (1099 Business)

In [1]:
import timeit
tic=timeit.default_timer()
import datetime
import calendar
import numpy as np
import pandas as pd
import snowflake.connector
import pandas.io.sql as sql

conn = snowflake.connector.connect(
                user='------',
                password='------',
                account='---------'
                )
cursor = conn.cursor()

data_tech = """SELECT a.SVC_PRT_STS_DT,a.SVC_UN_NO,a.SO_NO,a.PRT_ORD_QT,a.PRT_ORD_LN_STS_CD,a.prt_ord_ln_no,
a.svc_div_no,a.PRT_PRC_LIS_SRC_NO,a.ASL_CATEGORY,a.SVC_PRT_NO,a.PRT_PRM_PRC_AM,
LPAD(a.svc_div_no, 4, '0') || LPAD(a.prt_prc_lis_src_no, 3, '0') ||'_'|| 
a.svc_prt_no ||'_'|| e.CUSTSTATE AS item_id,
a.SO_NO || LPAD(a.svc_div_no, 4, '0') || LPAD(a.prt_prc_lis_src_no, 3, '0') AS so_div_pls,
a.PRT_DS,d.MDL_NO,d.BND_NM,e.SERVICEORDER,e.CUSTSTATE,c.EMPLOYEE_ID,c.EMPLOYEE_ENTERPRISEID,c.EMPLOYEE_NAME
FROM
PRD_NFDT.NFDT_PERM_TBLS.SERVICEORDER_PARTS a
JOIN PRD_DB2.HS_DW_VIEWS.NPSXTPY b
ON a.SVC_UN_NO=b.SVC_UN_NO
AND a.EMP_ID_NO=b.SVC_TEC_EMP_ID_NO
JOIN PRD_NFDT.NFDT_PERM_TBLS.TECHNICIAN_ROSTER_CURRENT c
on b.CRP_EMP_ID_NO=c.EMPLOYEE_ID 
JOIN PRD_DB2.HS_DW_VIEWS.NPSXTCL d
ON a.SVC_UN_NO=b.SVC_UN_NO and a.SO_NO= d.SO_NO 
JOIN PRD_NFDT.NFDT_PERM_TBLS.TRANSACTIONS e
ON e.SERVICEORDER=a.SO_NO
WHERE  a.SVC_UN_NO|| a.SO_NO in (select SVC_UN_NO|| SO_NO 
FROM PRD_NFDT.NFDT_PERM_TBLS.SERVICEORDER_PARTS where prt_seq_no >1 group by 1 )
AND a.SVC_PRT_STS_DT >= '2020-01-01'
"""
#Extract and load Tech data
query = pd.read_sql_query(data_tech, conn)

tech_data = query[query["CUSTSTATE"].isin(['GA','WI','FL','TX','LA','ND','CA','AR','CO','TN',
                                        'KS','NM','MO','NE','AZ','IL','OK','IA','NV','UT'])]
#query = query[query["ASL_CATEGORY"].isin(['TECH'])]
tech_data = tech_data[~query["SVC_DIV_NO"].isin(['099'])]
tech_data['SVC_PRT_STS_DT'] = pd.to_datetime(tech_data['SVC_PRT_STS_DT'], format='%Y-%m-%d', infer_datetime_format=True)
tech_data['Year'] = tech_data['SVC_PRT_STS_DT'].dt.year
tech_data['Month'] = tech_data['SVC_PRT_STS_DT'].dt.month
tech_data['monthName'] = tech_data['Month'].apply(lambda x: calendar.month_name[x])
tech_data['Month_Name'] = tech_data['monthName'].str[:3]
tech_data['Month_Year'] = tech_data['Month_Name'].map(str) + "_" + tech_data['Year'].map(str)

data2 = tech_data[['SVC_PRT_NO','SVC_DIV_NO','PRT_ORD_QT','PRT_PRM_PRC_AM','CUSTSTATE','PRT_DS','Month_Year']]
data2 = data2.groupby(['SVC_PRT_NO','SVC_DIV_NO','CUSTSTATE','PRT_DS','Month_Year']).agg(
    Qnt_Sold=pd.NamedAgg(column='PRT_ORD_QT', aggfunc='sum'),
    Revenue=pd.NamedAgg(column='PRT_PRM_PRC_AM', aggfunc='sum')
)
data2.reset_index(drop=False, inplace=True)

#Load and join tables needed for parts division description. 
conn = snowflake.connector.connect(
                user='bagabi',
                password='Ben@2020',
                account='sears_hs_prod.us-east-1'
                )
cursor = conn.cursor()
data3 = """SELECT a.svc_div_no,a.SVC_PRT_NO,LPAD(a.svc_div_no, 4, '0') || LPAD(a.prt_prc_lis_src_no, 3, '0') || 
a.svc_prt_no AS RECOMMENDED_PARTID, LPAD(a.svc_div_no, 4, '0') AS ID,
a.PRT_DS
FROM
PRD_NFDT.NFDT_PERM_TBLS.SERVICEORDER_PARTS a
Where a.SVC_PRT_STS_DT >= '2020-01-01'
"""
dat = """SELECT ID, NAME from PRD_KM.KM.DIVISION
"""
div = pd.read_sql_query(dat, conn)
service_part = pd.read_sql_query(data3, conn)

toc=timeit.default_timer()
Elapsed = toc - tic 
hours, rem = divmod(Elapsed, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

C:\Users\bagabi\AppData\Local\Continuum\anaconda3\lib\site-packages\snowflake\connector\options.py:78: UserWarning: You have an incompatible version of 'pyarrow' installed (0.15.1), please install a version that adheres to: 'pyarrow<0.18.0,>=0.17.0; extra == "pandas"'
  warn_incompatible_dep('pyarrow', _installed_pyarrow_version.version, _expected_pyarrow_version)
C:\Users\bagabi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:44: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


00:10:53.38


In [2]:
service_part_div = pd.merge(service_part, div)
div_desc = service_part_div[['SVC_PRT_NO','NAME']]
#div_desc.rename({'SVC_PRT_NO': 'PRT_NO'}, axis=1, inplace=True)
desc_div2 = div_desc.drop_duplicates(subset=['SVC_PRT_NO'], keep='first')
desc_div2.to_excel('Parts Division Name.xlsx',index=False)

dw = data2.copy()
dw['Month_Year'] = dw['Month_Year'].apply(lambda x: x.replace('_','')).map(str)
dw['YEAR'] = dw['Month_Year'].str[-4:]
dw['MONTH'] = dw['Month_Year'].str[:3]
dw = dw[['SVC_PRT_NO','PRT_DS','SVC_DIV_NO','Qnt_Sold','Revenue','CUSTSTATE','MONTH','YEAR']]
dw.sort_values("Qnt_Sold", inplace = True, ascending=False)
dw['Revenue'] = dw['Revenue'].map('${:,.2f}'.format)
dw.to_excel('Home Town 1099 Business Top Selling Parts.xlsx.xlsx',index=False)

In [3]:
dw.head()

,SVC_PRT_NO,PRT_DS,SVC_DIV_NO,Qnt_Sold,Revenue,CUSTSTATE,MONTH,YEAR
126967,TCA38091801,COMPRESSOR,046,2403,"$2,644,476.09",TX,Sep,2020
80771,ADH73150210,DRIER ASSY,046,2333,"$450,774.38",TX,Sep,2020
80770,ADH73150210,DRIER ASSY,046,2252,"$490,019.28",TX,Oct,2020
126966,TCA38091801,COMPRESSOR,046,2174,"$2,607,786.45",TX,Oct,2020
80383,ADH73150210,DRIER ASSY,046,2123,"$414,334.01",CA,Oct,2020
